*TP réalisé par Rachel Blin dans le cadre du cours d'Alexandrina Rogozan*

# Codage de Huffman

L'objectif de ce TP est de réaliser le codage de Huffman pour le message suivant :  
  
"*Il n'existe que deux choses infinies, l'univers et la bêtise humaine... mais pour l'univers, je n'ai pas de certitude absolue.*" (Albert Einstein)

## Simplification de la séquence

Afin de rendre la séquence plus simple à encoder, nous allons tout d'abord transformer la phrase de telle sorte à ce qu'elle ne contienne que des caractères présents dans les 26 lettres de l'alphabet en minuscule, sans accents, ainsi que les espaces.

La message à encoder devient donc :  
  
"*il nexiste que deux choses infinies lunivers et la betise humaine mais pour lunivers je nai pas de certitude absolue*

## Classification des symboles de la séquence par ordre d'occurences croissants

La première étape du codage de Huffman est de classer les symboles de la séquence à encoder par nombre d'occurences croissants.  
  
1) Dans un premier temps, répertoriez tous les caractères présents dans le message.

In [5]:
m = "il nexiste que deux choses infinies lunivers et la betise humaine mais pour lunivers je nai pas de certitude absolue"

def unique(message):
    """ Returs all the uninque char in a string.
    
    # Argument:
        - message: The string to extract the unique characters from
        
    # Returns:
        - A list containing all the unique characters in the input string.
    """
    chars = [] 
    for char in message: 
        if char not in chars: 
            chars.append(char) 
    return chars

chars = unique(m)
print(chars)

['i', 'l', ' ', 'n', 'e', 'x', 's', 't', 'q', 'u', 'd', 'c', 'h', 'o', 'f', 'v', 'r', 'a', 'b', 'm', 'p', 'j']


2) Calculez maintenant le nombre d'occurences de chacun de ces caractères dans le message et rangez ces occurences par nombre croissant.

In [6]:
import operator
import collections

def occurences(message):
    """ Counts the number of occurencies in a given message.
    
    # Argument:
        - message: The string from which we wish to count the occurencies of the chars.
    
    # Returns:
        - A dict containing the chars as keys and the number of occurencies of each chars.
    """
    counts = []
    chars = unique(message)
    for char in chars:
        nb = message.count(char)
        counts.append(nb)
    occurences = dict(zip(chars, counts))
    return occurences

def order_by_values(dictionary):
    """ Helper function to order a dictionnary by its values.
    
    # Argument:
        - dictionary: The dictionary to order.
        
    # Returns:
        - An ordered list of tuples containing as first value the character and as second the number of occurencies
    
    """
    return sorted(dictionary.items(), key=operator.itemgetter(1))

occurences = occurences(m)
print(occurences)
sorted_occurences = order_by_values(occurences)
print(sorted_occurences)

{'q': 1, 'u': 8, 's': 10, 'd': 3, 't': 5, ' ': 19, 'n': 7, 'x': 2, 'm': 2, 'h': 2, 'a': 6, 'e': 17, 'v': 2, 'i': 12, 'l': 5, 'o': 3, 'b': 2, 'j': 1, 'c': 2, 'f': 1, 'p': 2, 'r': 4}
[('q', 1), ('j', 1), ('f', 1), ('x', 2), ('m', 2), ('h', 2), ('v', 2), ('b', 2), ('c', 2), ('p', 2), ('d', 3), ('o', 3), ('r', 4), ('t', 5), ('l', 5), ('a', 6), ('n', 7), ('u', 8), ('s', 10), ('i', 12), ('e', 17), (' ', 19)]


## Séparation des symboles en deux sous-groupes

Une fois les symboles classés par ordre d'occurence décroissante, on va maintenant créer un arbre dont les feuilles sont les cractères présents dans le message avec leur nombre d'occurences dans le message. Afin de créer cet arbre, on va d'abord chercher les noeuds ayant le plus petit nombre d'occurences et les accrocher à un noeud dont l'occurence est la somme des occurences des deux noeuds. On va répéter cette opération de sorte à ce qu'il n'y ait plus qu'un seul noeud dans l'arbre.

__Exemple__ :  
Pour une séquence contenant les caractères (a, b, c, d, e) d'occurences respectives (1, 1, 2, 2, 3) on obtient le graphe suivant : 

![graphe](TP_Huffman.png)

Ce qui donne donc le codage de Huffman suivant : 

| Caractère | Nombre d'occurences | Code Huffman |
|-----------|---------------------|-------------------|
| a         | 1                   | 111               |
| b         | 1                   | 110               |
| c         | 2                   | 10                |
| d         | 2                   | 01                |
| e         | 3                   | 00                |

3) Créez le graphe de séparation des symboles.

In [10]:
# Création d'une structure de données de type arbre
class Tree(object):
    """ Tree object."""
    def __init__(self, g=None, d=None, data=None):
        """ Init function.
        
        # Arguments:
            - g: The left node of the tree.
            - d: The right node of the tree.
            - data: The data to be held in the node.
        """
        self.g = g
        self.d = d
        self.data = data

def update_list_node(list_node, new_node):
    """ Adds a node in the correct place in an ordered list of nodes.
    
    # Arguments:
        - list_node: The list of nodes we want to add the node to.
        - new_node: The node to be added.
    
    # Returns:
        - The list with the added node.
    """
    for index, node in enumerate(list_node):
        if node.data[1] >= new_node.data[1]:
            list_node[index:index] = [new_node]
            break
    else:
        list_node.append(new_node)
            
    return list_node

def Huffman(sorted_occurences):
    """ Creates the Huffman tree.
    
    # Argument:
        - sorted_occurences: The sorted occurencies of the message we wish to encode.
    
    # Returns:
        - The root of the Huffman tree.
    """
    list_node = [Tree(data=value) for value in sorted_occurences]
    
    while(len(list_node) > 1):
        node_left = list_node.pop(0)
        node_right = list_node.pop(0)
        new_data = (node_left.data[0] + node_right.data[0], node_left.data[1] + node_right.data[1])
        new_node = Tree(node_left, node_right, new_data)
        update_list_node(list_node, new_node)

    return list_node[0]
    

def display_graph(root_node):
    current_level = [root_node]
    nb_nodes = ""
    level = 0
    while current_level:
        next_level = list()
        nb_nodes = nb_nodes[:len(nb_nodes)//2]
        for node in current_level:
            print(nb_nodes + str(node.data))
            if node.g:
                next_level.append(node.g)
            if node.d:
                next_level.append(node.d)
            print("level : ", level)
            current_level = next_level
        level +=1
        
def get_dict_codage(graph):
    """ Creates the coding dictionnary.
    
    # Argument:
        - graph: The root of the Huffman tree.
    
    # Returns:
        - A coding dictionary.
    """
    current_level = [graph]
    nb_nodes = ""
    level = 0
    dict_Huffman = dict()
    while current_level:
        next_level = list()
        nb_nodes = nb_nodes[:len(nb_nodes)//2]
        for node in current_level:
            if node.g:
                next_level.append(node.g)
                seq = node.g.data[0]
                for char in seq:
                    if char in dict_Huffman:
                        dict_Huffman[char] += "1"
                    else:
                        dict_Huffman[char] = "1"
            if node.d:
                next_level.append(node.d)
                seq = node.d.data[0]
                for char in seq:
                    if char in dict_Huffman:
                        dict_Huffman[char] += "0"
                    else:
                        dict_Huffman[char] = "0"
            current_level = next_level
        level +=1
    return dict_Huffman

sorted_occurences = order_by_values(occurences)
Huffman_graph = Huffman(sorted_occurences)

print("Affichage du graphe de séparation des symboles :")
display_graph(Huffman_graph)
print("\n")
dict_Huffman = get_dict_codage(Huffman_graph)
print("Affichage du dictionnaire des encodages de Huffman par caractère :")
print(dict_Huffman)
        

Affichage du graphe de séparation des symboles :
('sldoianxmrbchveupfqjt ', 116)
level :  0
('sldoian', 46)
level :  1
('xmrbchveupfqjt ', 70)
level :  1
('sldo', 21)
level :  2
('ian', 25)
level :  2
('xmrbchve', 33)
level :  2
('upfqjt ', 37)
level :  2
('s', 10)
level :  3
('ldo', 11)
level :  3
('i', 12)
level :  3
('an', 13)
level :  3
('xmrbchv', 16)
level :  3
('e', 17)
level :  3
('upfqjt', 18)
level :  3
(' ', 19)
level :  3
('l', 5)
level :  4
('do', 6)
level :  4
('a', 6)
level :  4
('n', 7)
level :  4
('xmr', 8)
level :  4
('bchv', 8)
level :  4
('u', 8)
level :  4
('pfqjt', 10)
level :  4
('d', 3)
level :  5
('o', 3)
level :  5
('xm', 4)
level :  5
('r', 4)
level :  5
('bc', 4)
level :  5
('hv', 4)
level :  5
('pfqj', 5)
level :  5
('t', 5)
level :  5
('x', 2)
level :  6
('m', 2)
level :  6
('b', 2)
level :  6
('c', 2)
level :  6
('h', 2)
level :  6
('v', 2)
level :  6
('p', 2)
level :  6
('fqj', 3)
level :  6
('f', 1)
level :  7
('qj', 2)
level :  7
('q', 1)
level :  8
('

4) Effectuez maintenant le codage de Huffman.

In [11]:
def encode_message(m, dict_Huffman):
    """ Encodes a given message.
    
    # Arguments:
        - m: The message to encode.
        - dict_Huffman: The coding dictionary.
    
    # Returns:
        - The encoded message.
    """
    message_Huffman = ""
    for char in m:
        message_Huffman = message_Huffman + dict_Huffman[char]
    return message_Huffman

message_Huffman = encode_message(m, dict_Huffman)
print(message_Huffman)

1011101000100001001111110111100100010000001010010011010000110010100011011111000011010011001110001110101110001011000001010110110001010101110001101001110001010110000100111011100001000100000110110010000110110100010010111101000001100100110111101001101100001000001111010011011110000010111100000110111000011010011100010101100001001110111000001010000100001000100110100000101110011110001100101000001101001001110001001010010000111100101000010010110111111100011010011010


## Visualisation de l'encodage 

En se basant sur le codage binaire des caractères de la table ASCII, le message de départ est le suivant :  
"0100100101001100001000000100111001000101010110000100100101010011010101000100010100100000010100010101010101000101001000000100010001000101010101010101100000100000010000110100100001001111010100110100010101010011001000000100100101001110010001100100100101001110010010010100010101010011001000000100110001010101010011100100100101010110010001010101001001010011001000000100010101010100001000000100110001000001001000000100001001000101010101000100100101010011010001010010000001001000010101010100110101000001010010010100111001000101001000000100110101000001010010010101001100100000010100000100111101010101010100100010000001001100010101010100111001001001010101100100010101010010010100110010000001001010010001010010000001001110010000010100100100100000010100000100000101010011001000000100010001000101001000000100001101000101010100100101010001001001010101000101010101000100010001010010000001000001010000100101001101001111010011000101010101000101"    

Pour rappel, le message encodé par la méthode de Shannon-Fano est le suivant :    

"10010011110101101000001100110000100010111000000000111011100100010101100000111000010000011001001100010110001110010101000000011001010110011011000110011011010110010001001010010110001110101000110011010011100010110101000100110001011100001101100011001001100101011011100011001001100110001100011100100101100101110011011010110010001001010010110001100000011011101010100110011100011101001100011001000101110000101010010101000100101000011001000101110100100010110000010010011011101"    

5) Que peut-on dire du rôle du codage de Huffman? Est-il plus ou moins efficace que le codage de Shannon-Fano? Vous calculerez le taux de compression pour illustrer votre réponse.

In [12]:
binaire = "0100100101001100001000000100111001000101010110000100100101010011010101000100010100100000010100010101010101000101001000000100010001000101010101010101100000100000010000110100100001001111010100110100010101010011001000000100100101001110010001100100100101001110010010010100010101010011001000000100110001010101010011100100100101010110010001010101001001010011001000000100010101010100001000000100110001000001001000000100001001000101010101000100100101010011010001010010000001001000010101010100110101000001010010010100111001000101001000000100110101000001010010010101001100100000010100000100111101010101010100100010000001001100010101010100111001001001010101100100010101010010010100110010000001001010010001010010000001001110010000010100100100100000010100000100000101010011001000000100010001000101001000000100001101000101010100100101010001001001010101000101010101000100010001010010000001000001010000100101001101001111010011000101010101000101"

message_Shanon_Fano = "10010011110101101000001100110000100010111000000000111011100100010101100000111000010000011001001100010110001110010101000000011001010110011011000110011011010110010001001010010110001110101000110011010011100010110101000100110001011100001101100011001001100101011011100011001001100110001100011100100101100101110011011010110010001001010010110001100000011011101010100110011100011101001100011001000101110000101010010101000100101000011001000101110100100010110000010010011011101"

print("longueur du message en binaire : ", len(binaire))
print("longueur du message codage de Shannon-Fano : ", len(message_Shanon_Fano))
print("longueur du message codage de Huffman : ", len(message_Huffman))

def taux_compression(longueur_source, longueur_encode):
    taux = 1 - (longueur_encode / longueur_source)
    return taux

taux_Shannon_Fano = taux_compression(len(binaire), len(message_Shanon_Fano))
taux_Huffman = taux_compression(len(binaire), len(message_Huffman))
print("Le taux de compression du codage de Shannon-Fano pour le message est : ", taux_Shannon_Fano)
print("Le taux de compression du codage de Huffman pour le message est : ", taux_Huffman)

longueur du message en binaire :  928
longueur du message codage de Shannon-Fano :  467
longueur du message codage de Huffman :  460
Le taux de compression du codage de Shannon-Fano pour le message est :  0.4967672413793104
Le taux de compression du codage de Huffman pour le message est :  0.5043103448275862


On constate que le codage de Huffman réduit la taille du message à envoyer par rapport au codage de Shanon-Fano. En effet, le taux de compression du codage de Huffman est plus élevé que celui de Shannon-Fano pour le message.